## Example of computing the photon-noise limited RV precision for a single star with a particular RV spectrograph 

The following is an example of how to calculate the photon-noise limited RV precision using the formalism from http://adsabs.harvard.edu/abs/2001A%26A...374..733B and the python code outlined in http://adsabs.harvard.edu/abs/2018AJ....156...82C

In [1]:
from compute_sigmaRV import *


Bad key "xtick.labeltop" on line 358 in
/Users/ryancloutier/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [11]:
# Get an approximate telluric spectrum to mask regions of high contamination. 
# This may take a few seconds to load but only needs to be done once.
wl_telluric, trans_telluric = np.loadtxt('./InputData/tapas_000001.ipac', skiprows=23).T
wl_telluric *= 1e-3  # microns


def compute_sigRV_phot(band_strs, mags, Teff, logg, Z, vsini, texp, R, aperture, throughput, RVnoisefloor,
                       transmission_threshold=.02):
    '''
    Calculate the photon-noise limited RV precision over the spectrograph's full spectral domain.
    '''
    if 'V' in band_strs:
        centralwl_nm = 555.
    elif 'J' in band_strs:
        centralwl_nm = 1250.
    else:
        print band_strs
        raise ValueError('One or both of V and J must be in band_strs.')
    
    # get round values for PHOENIX stellar models
    Teffs = np.append(np.arange(23e2,7e3,1e2), np.arange(7e3,121e2,2e2))
    Teff_round = Teffs[abs(Teffs-Teff) == np.min(abs(Teffs-Teff))][0]
    loggs = np.arange(0, 6.1, .5)
    logg_round = loggs[abs(loggs-logg) == np.min(abs(loggs-logg))][0]
    Zs = np.append(np.arange(-4,-1,dtype=float), np.arange(-1.5,1.5,.5))
    Z_round = Zs[abs(Zs-Z) == np.min(abs(Zs-Z))][0]

    # compute sigmaRV in each band for a fixed texp
    mags = np.ascontiguousarray(mags)
    sigmaRVs, SNRtargets = np.zeros(mags.size), np.zeros(mags.size)
    for i in range(sigmaRVs.size):
        t0 = time.time()
        SNRtargets[i] = get_snr(mags[i], band_strs[i], texp, aperture, throughput, R)
        wl, spec = get_reduced_spectrum(Teff_round, logg_round, Z_round, vsini,
                                        band_strs[i], R, centralwl_nm*1e-3,
                                        SNRtargets[i])
        sigmaRVs[i] = compute_sigmaRV(wl, spec, mags[i], band_strs[i], texp,
                                      aperture, throughput, R,
                                      transmission_threshold, wl_telluric,
                                      trans_telluric, SNRtargets[i])

    # compute sigmaRV over all bands
    sigRV_phot = 1 / np.sqrt(np.sum(1. / sigmaRVs**2))
    sigRV_phot = sigRV_phot if sigRV_phot > RVnoisefloor \
                 else float(RVnoisefloor)
    return sigmaRVs, sigRV_phot

First, define the following parameters of the RV spectrograph (and telescope) used to conduct the RM observations.

In [25]:
Teff = 4500      # stellar effective temperature in Kelvin
logg = 4.5       # stellar surface gravity in dex (cgs) 
Z = 0            # stellar metallicity [Fe/H] in solar units
vsini = .01        # projected stellar rotation velocity in km/s
band_strs = ['B','V','R'] # names of spectrograph spectral bands. Must be in ['U','B','V','R','I','Y','J','H','K']

Next, define the following stellar parameters for your star. Here I'll use an arbitrary (made up) star as an example.

In [26]:
texp = 8        # exposure time in seconds
R = 5e4          # spectral resolution == lambda / delta_lambda
aperture = 1.2     # telescope aperture diameter in meters
throughput = .015  # average spectrograph throughput (in [0,1])
RVfloor = .01     # RV noise floor in m/s
mags = [6.7, 6, 5.6]  # stellar magnitudes for each band in `band_strs'
assert len(band_strs) == len(mags)

Now we are in a position to compute the photon-noise limited RV precision of our star with our parameterized observational setup. The calculation should only take a few seconds depending on the number of bands and the complexity of the RV information content in those bands.

In [27]:
theta = band_strs, mags, Teff, logg, Z, vsini, texp, R, aperture, throughput, RVfloor
sigRVs, sigRV = compute_sigRV_phot(*theta)
print '\nPhoton-noise limited RV precision = %.3f m/s'%sigRV

Convolving the B band stellar spectrum to the instrumental resolution (R = 50000)
Masking B band telluric regions where transmission > 0.98 percent
Convolving the V band stellar spectrum to the instrumental resolution (R = 50000)
Masking V band telluric regions where transmission > 0.98 percent
Convolving the R band stellar spectrum to the instrumental resolution (R = 50000)
Masking R band telluric regions where transmission > 0.98 percent

Photon-noise limited RV precision = 1.862 m/s


In [23]:
sigRVs

array([32.8715704 ,  3.22923057,  2.65490106,  2.0686954 ])

In [28]:
3e8/189/1.862

852471.3143402723

In [29]:
3e8/189/852471.

1.862000686594133